In [1]:
from kafka import KafkaProducer, KafkaConsumer
from kafka import KafkaClient
import os
import io
import random
import avro.schema
from avro.io import DatumWriter
import json
import uuid
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.avro.functions import from_avro, to_avro

SCALA_VERSION = '2.12'
SPARK_VERSION = '3.2.1'


os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64/'
os.environ["SPARK_HOME"] = "C:\spark\spark-3.2.1-bin-hadoop3.2"
os.environ["HADOOP_HOME"] = "C:\spark\spark-3.2.1-bin-hadoop3.2\hadoop"
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

findspark.init('/home/camilli/.local/lib/python3.8/site-packages/pyspark')

In [2]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkByExamples.com') \
                    .getOrCreate()

print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

topic = 'operacoes'
topic_enriquecido = 'test2'
bootstrapServers = 'localhost:9092'

# `from_avro` requires Avro schema in JSON string format.
jsonFormatSchema = open("operacoes.avsc", "r").read()

# .config('spark.jars.packages', 'spark-sql-kafka-0-10_2.13,spark-avro_2.13')\

22/07/22 03:59:10 WARN Utils: Your hostname, BlackBerry resolves to a loopback address: 127.0.1.1; using 172.17.174.206 instead (on interface eth0)
22/07/22 03:59:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/camilli/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/camilli/.ivy2/cache
The jars for the packages stored in: /home/camilli/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a633627d-a8ac-4ed2-bf15-ab07e0ac97e7;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.1 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in centr

PySpark Version :3.1.2
PySpark Version :3.1.2


In [3]:
import mysql.connector
import pandas as pd

df = spark\
  .readStream\
  .format("kafka")\
  .option("kafka.bootstrap.servers", bootstrapServers)\
  .option("subscribe", topic)\
  .option("startingOffsets", "earliest") \
  .load()

#df.selectExpr("CAST(value AS STRING) as json")
# output = df\
#   .select(from_avro("value", jsonFormatSchema).alias("operations"))

# query = df \
#     .writeStream \
#     .format("csv") \
#     .option("path","/home/camilli/projetos/checkpoint") \
#     .option("checkpointLocation", "/home/camilli/projetos/checkpoint") \
#     .start()

# df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
#             .writeStream \
#             .format("kafka") \
#             .option("kafka.bootstrap.servers", bootstrapServers) \
#             .option("subscribe", topic)\
#             .option("checkpointLocation", "/home/camilli/projetos/checkpoint") \
#             .start()

# df.selectExpr("CAST(value AS STRING)", "CAST(timestamp AS TIMESTAMP)") \

def enriquecimento(df, batch_id):
  df_operacoes = df.rdd.map(lambda x: x.value).collect()
  print("passei por aqui")
  df_operacoes.show()

  conn = mysql.connector.connect(database='spark',
                               host="localhost",
                               user='root',
                               password='root')

  db_cursor = conn.cursor()
  select = 'Select idClient, name, birth, accManager, Account, score from enrichment'
  db_cursor.execute(select)
  rows = db_cursor.fetchall()
  df_db = pd.DataFrame(rows)
  conn.close()

  df_cliente = spark.createDataFrame(df_db)
  df_cliente.show()

  df_enr = df_operacoes.join(df_cliente, df_operacoes.codigoCliente==df_cliente.idClient,"inner")
  
  df_enr = df_enr.drop("codigoCliente")

  df_enr.withColumn("result",
       when((col("tipOperacao") == "saque") & (col("valOperacao") > col("saldoEmConta")), "Negado")
      .otherwise("Aprovado"))

  df_enr.withColumn("Credito", when((col("score") >= "700"), True).otherwise(False))

  df_enr.write.format('jdbc').options(
      url='jdbc:mysql://localhost:3306/spark',
      driver='com.mysql.jdbc.Driver',
      dbtable='operacao',
      user='root',
      password='root').mode('append').save()



df1 = df.select(from_avro("value", jsonFormatSchema).alias("operations"))\
        .writeStream \
        .format("console") \
        .foreachBatch(enriquecimento) \
        .outputMode("append") \
        .trigger(processingTime="5 second") \
        .option("checkpointLocation", "/home/camilli/projetos/checkpoint") \
        .option("spark.sql.streaming.forceDeleteTempCheckpointLocation","true")\
        .start() 



22/07/22 03:59:24 ERROR MicroBatchExecution: Query [id = 134f8781-6620-46b1-aa72-6846eb37f1fe, runId = 73e4f50d-95d2-4619-b757-bed202e08bf7] terminated with error
java.lang.NoClassDefFoundError: org/apache/spark/sql/connector/read/streaming/SupportsTriggerAvailableNow
	at java.lang.ClassLoader.defineClass1(Native Method)
	at java.lang.ClassLoader.defineClass(ClassLoader.java:756)
	at java.security.SecureClassLoader.defineClass(SecureClassLoader.java:142)
	at java.net.URLClassLoader.defineClass(URLClassLoader.java:473)
	at java.net.URLClassLoader.access$100(URLClassLoader.java:74)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:369)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:363)
	at java.security.AccessController.doPrivileged(Native Method)
	at java.net.URLClassLoader.findClass(URLClassLoader.java:362)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at sun.misc.Launcher$AppClassLoader.loadClass(Launcher.java:352)
	at java.lang.ClassLoader.loadClass(ClassLoad